In [ ]:
"""
Takes [hml_filename].hml
and converts them into csv format

Beware of overwriting
Choose the file name (source and dest) carefully

Beware of duplicated / missing / combined data records
(especially on appended records in the same file)
In hml_to_csv, the default is to convert the last record on the file

Beware of the enabled metrics, keep them all the same throughout the file
Multiple records with diferrent metrics WILL conflict

The code will try to read the GPU name from the log file,
overwrite the name if needed

Also edit app name

Errors -> Good luck
"""

import csv
import datetime
from itertools import islice
from pathlib import Path

# Edit the path and filenames
filename = "/content/HardwareMonitoring.hml"

# Overwrite the columns in csv if not empty
# Edit app_name if the .exe file name is as clear as mud to the reader
gpu_name = ""
app_name = "Wuthering Waves"


In [ ]:
def hml_to_csv(src, dest, gpu_name='', app_name='', convert_at_records: int|list[int]|range|str=-1, encoder='cp1252'):

    hml_path = Path(src).with_suffix('.hml')
    if not hml_path.exists():
        raise FileNotFoundError()

    print(f"Received: {hml_path}")
    start_record_rows = []
    with open(hml_path, 'r', newline='', encoding=encoder) as infile:
        # Read the file twice, one for finding the starting line numbers of the records,
        # and then actually write the data.
        reader = csv.reader(infile)
        for rownumber, row in enumerate(reader):
            if row and (row[0] == '01'):
                start_record_rows.append(rownumber)

    began = False
    convert_start_rows = []
    if isinstance(convert_at_records, int):
        convert_start_rows.append(start_record_rows[convert_at_records])
    elif isinstance(convert_at_records, str) and convert_at_records == 'all':
        convert_start_rows = start_record_rows
    else:
        for rownumber in iter(convert_at_records):
            convert_start_rows.append(start_record_rows[rownumber])

    use_file_index = False
    if len(convert_start_rows) > 1:
        use_file_index = True

    print(f"Found {len(start_record_rows)} records.")
    print(f"Converting to {len(convert_start_rows)} file(s)...")
    print()

    for convert_number, convert_start_row in enumerate(convert_start_rows):
        began = False
        data_rows = []
        headers = []
        units = []

        # start reading from ...
        with open(hml_path, 'r', newline='', encoding=encoder) as infile:
            reader = csv.reader(infile)
            for row in islice(reader, convert_start_row, None):
                if row[0] == '01':
                    # Extract GPU name (first GPU if multiple)
                    if not began:
                        began = True
                        if not gpu_name:
                            gpu_name = row[2].split(',')[0].strip()
                    else:
                        # Hit a different record
                        break
                elif row[0] == '02':
                    headers = row[2:]
                elif row[0] == '03':
                    units.append(row[3].strip())
                elif row[0] == '80':
                    data_rows.append(row[1:])


        combined_headers = ["GPU", "Application", "Timestamp"] + [f"{h.strip()} ({u})" if u else h.strip() for h, u in zip(headers, units)]
        this_result_path = dest
        if use_file_index:
            this_result_path += f'_{convert_number}'
        this_result_path = Path(this_result_path).with_suffix('.csv')

        with open(this_result_path, 'w', newline='', encoding=encoder) as outfile:
            writer = csv.writer(outfile)
            writer.writerow(combined_headers)
            for row in data_rows:
                writer.writerow([cell.strip() for cell in [gpu_name, app_name] + row])


        dtformat = r"%d-%m-%Y %H:%M:%S"
        start_time = datetime.datetime.strptime(data_rows[0][0].strip(), dtformat)
        end_time = datetime.datetime.strptime(data_rows[-1][0].strip(), dtformat)

        print(f"Record #{convert_number} | Starting at: {start_time}")
        print(f"Converted to: {this_result_path}")
        print(f"Record time: {end_time-start_time} | Total lines converted: {len(data_rows)}")
        print()


In [ ]:
hml_to_csv(
    src= filename,
    dest = filename,
    gpu_name=gpu_name,
    app_name=app_name,
    convert_at_records='all')


Received: /content/HardwareMonitoring.hml
Found 1 records.
Converting to 1 file(s)...

Record #0 | Starting at: 2026-01-10 17:05:26
Converted to: /content/HardwareMonitoring.csv
Record time: 0:05:10 | Total lines converted: 311

